In [ ]:
import os
import shutil
from icrawler.builtin import GoogleImageCrawler
import random

# Parametri
labels = ['track', 'motorcycle', 'bicycle', 'car', 'tram', "train", "bus"]
num_training_labels = 3  # Quanti animali vanno nel training set
max_images_per_label = 50
base_dir = 'testing_images2'
training_dir = os.path.join(base_dir, 'training')
query_dir = os.path.join(base_dir, 'test/query')
gallery_dir = os.path.join(base_dir, 'test/gallery')

# Crea la struttura delle cartelle
for path in [training_dir, query_dir, gallery_dir]:
    os.makedirs(path, exist_ok=True)

# Mescola e dividi le label
random.shuffle(labels)
training_labels = labels[:num_training_labels]
test_labels = labels[num_training_labels:]

print(f"Training labels: {training_labels}")
print(f"Test labels: {test_labels}")

# Funzione per scaricare immagini per ogni animale
def download_images(label, output_dir):
    crawler = GoogleImageCrawler(storage={'root_dir': output_dir})
    crawler.crawl(keyword=label, max_num=max_images_per_label)

# --- Sezione TRAINING ---
for label in training_labels:
    print(f"[TRAINING] Scarico immagini per: {label}")
    label_dir = os.path.join(training_dir, label)
    os.makedirs(label_dir, exist_ok=True)
    download_images(label, label_dir)

# --- Sezione TEST (query + gallery) ---
for label in test_labels:
    print(f"[TEST] Scarico immagini per: {label}")
    temp_dir = f"temp_{label}"
    os.makedirs(temp_dir, exist_ok=True)
    download_images(label, temp_dir)

    # Lista immagini scaricate
    images = [os.path.join(temp_dir, img) for img in os.listdir(temp_dir) if img.endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(images)

    # Seleziona una immagine come query e 5 per gallery
    if len(images) >= 6:
        shutil.copy(images[0], os.path.join(query_dir, f"{label}_query.jpg"))
        for i, img in enumerate(images[1:6]):
            shutil.copy(img, os.path.join(gallery_dir, f"{label}_gallery_{i}.jpg"))

    # Rimuovi temp dir
    shutil.rmtree(temp_dir)

print("\n✅ Download e organizzazione completati.")
